# PRÁCTICA 8: REGRESIÓN LINEAL, MÚLTIPLE Y LOGÍSTICA


## CQFidalgo

In [2]:
import os, zipfile, urllib.request

In [3]:
path = os.getcwd()
if (os.path.exists(path+'/avila.zip')):
    dataset = path+'/avila.zip'
else:
    url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip'
    dataset = urllib.request.urlretrieve(url, path+'/avila.zip')
    dataset = path+'/avila.zip'

In [4]:
file_zip = zipfile.ZipFile(dataset, 'r')
try:
    files = file_zip.namelist()
    file_zip.extractall(path = path+'/')
except:
    pass
file_zip.close()

# Apartado 1: 
Descargue el conjunto de datos AVILA del repositorio UCI. Verá que se encuentra
dividido en dos ficheros (trial y test). Júntelos en un sólo y añada una cabecera F1,F2….,
F10, Class. El resultado grábelo en un fichero para ser leído por el módulo pandas de
python.

In [5]:
import pandas as pd

In [6]:
trial = pd.read_csv(files[1], header = None)
test = pd.read_csv(files[2], header = None)

In [7]:
data = pd.concat([trial,test]) #Junta el contenido de los dos ficheros
data.columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Class'] #Añade la cabecera
data.to_csv(path+'/avila.csv', index = False) #Graba el resultado en un fichero csv
data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,Class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
10432,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
10433,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
10434,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
10435,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


# Apartado 2
Con este fichero generado realice una clasificación mediante regresión lineal múltiple.
Para ello, se destinará ⅔ de los datos escogidos aleatoriamente de manera estratificada
para aprendizaje y, el resto, para verificación.

In [8]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [12]:
file = pd.read_csv('avila.csv')

data = np.asarray(file)

X = data[:,:10]
X = MinMaxScaler().fit_transform(X) #Escala los datos
Y = data[:,-1]
Y

array(['A', 'A', 'A', ..., 'A', 'F', 'H'], dtype=object)

In [16]:
print(Y)

['A' 'A' 'A' ... 'A' 'F' 'H']


### Hold-Out

In [11]:
Z = LabelBinarizer().fit(np.unique(Y)).transform(Y) #Convierte a binario los valores de la clase
Z

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Z, test_size= 1./3., random_state=0, stratify=Y)

In [11]:
Y_predict = np.zeros(Y_test.shape, dtype=float) #Crea un array

for i in range(Y_test.shape[1]):
    regresion = LinearRegression().fit(X_train, Y_train[:,i]) #Ajusta el clasificador entrenandolo con la columna correspondiente
    Y_predict[:,i] = regresion.predict(X_test) #Asigna la prediccion a la posición correspondiente del array creado

In [12]:
print("Tasa de acierto: ", accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)))

Tasa de acierto:  0.4798734905117884


In [13]:
print("\n\t\t\tMatriz de Confusión:\n\n", confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)))


			Matriz de Confusión:

 [[2792    0    0    0    6   32    0    0   27    0    0    0]
 [   0    0    0    0    0    0    0    0    3    0    0    0]
 [  56    0    0    0    4    3    0    0    6    0    0    0]
 [ 224    0    0    0    1    1    0    0    9    0    0    0]
 [ 675    0    0    0   18   21    0    0   15    0    1    0]
 [1285    0    0    0    0    7    0    0   16    0    0    0]
 [ 290    0    0    0    1    3    0    0    4    0    0    0]
 [ 286    0    0    0    2   29    0    0   29    0    0    0]
 [  66    0    0    0    4    4    0    0  475    0    5    0]
 [  27    0    0    0    3    0    0    0    0    0    0    0]
 [ 208    0    0    0   13    0    0    0   81    0   46    0]
 [  42    0    0    0    1    1    0    0  134    0    0    0]]


# Apartado 3
Repetir el paso anterior empleando el método de validación cruzada con estratificación.
Compare los resultados.

### Cross Validation (10 particiones)

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
skf = StratifiedKFold(n_splits=10) #Crea las 10 particiones para la validación cruzada
accuracy = 0
for train_index, test_index in skf.split(X, Y):
    # Crea, entrena y predice según los 12 clasificadores en las 10 particiones
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Z[train_index], Z[test_index]

    Y_predict = np.zeros(Y_test.shape, dtype=float) #Crea un array

    for i in range(Y_test.shape[1]):
        regresion = LinearRegression().fit(X_train, Y_train[:,i]) #Ajusta el clasificador entrenandolo con la columna correspondiente
        Y_predict[:,i] = regresion.predict(X_test) #Asigna la prediccion a la posición correspondiente del array creado
    
    accuracy = accuracy + accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)) #Suma la tasa de acierto en cada iteración

accuracy = accuracy/10 #Calcula la tasa de acierto global

In [16]:
print("Tasa de acierto: ", accuracy)

Tasa de acierto:  0.4814777688296402


# Apartado 4

Repita los pasos 2 y 3 aplicando la Regresión logística. En ambos casos, solamente
cuando emplee el método del resorte o Hold-Out, calcule la matriz de confusión.

In [17]:
from sklearn.linear_model import LogisticRegression

### Hold-Out

In [18]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size= 1/3, random_state=1, stratify=Y)

In [19]:
clf = LogisticRegression(n_jobs=-1) #Crea el clasificador
clf.fit(X_train, Y_train) #Entrena el clasificador

Y_predict = clf.predict(X_test) #Crea las prediciones en base al clasificador ajustado
accuracy = clf.score(X_test, Y_test) #Calcula la tasa de acierto

In [20]:
print("Tasa de acierto: ", accuracy)

Tasa de acierto:  0.49511213341000576


In [21]:
print("\n\t\t\tMatriz de Confusión:\n", confusion_matrix(Y_test, Y_predict))


			Matriz de Confusión:
 [[2804    0    0    0    6   45    0    0    0    0    2    0]
 [   3    0    0    0    0    0    0    0    0    0    0    0]
 [  62    0    0    0    0    7    0    0    0    0    0    0]
 [ 226    0    0    0    0    8    0    1    0    0    0    0]
 [ 711    0    0    0    0   19    0    0    0    0    0    0]
 [1290    0    0    0    4    7    0    3    4    0    0    0]
 [ 293    0    0    0    0    0    0    1    0    0    4    0]
 [ 305    0    0    0    0   35    0    0    6    0    0    0]
 [  47    0    0    0    1    6    0    7  490    0    3    0]
 [  30    0    0    0    0    0    0    0    0    0    0    0]
 [ 198    0    0    0    3    4    0    0    0    0  143    0]
 [ 120    0    0    0    2    0    0    0   25    0   31    0]]


### Cross Validation

In [22]:
skf = StratifiedKFold(n_splits=10) #Crea las 10 particiones para la validación cruzada
accuracy = 0

for train_index, test_index in skf.split(X, Y):
    # Crea, entrena y predice según los 12 clasificadores en las 10 particiones
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    clf = LogisticRegression(n_jobs=-1) #Crea el clasificador
    clf.fit(X_train, Y_train) #Entrena el clasificador

    accuracy = accuracy + clf.score(X_test, Y_test)  #Suma la tasa de acierto en cada iteración

accuracy = accuracy/10 #Calcula la tasa de acierto total

In [23]:
print("Tasa de acierto: ", accuracy)

Tasa de acierto:  0.49753192961404225
